In [9]:
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [14]:
# Load the SQL magic
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
# Connect to a SQLite database (will create if not exists)
%sql sqlite:///chicago_data.db

In [19]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('chicago_data.db')
cursor = conn.cursor()

# Load and filter SQL script
with open("chicago_socioeconomic_data.sql", "r") as f:
    sql_script = f.read()

# Clean comments and MySQL-specific commands
cleaned_statements = []
for stmt in sql_script.split(';'):
    stmt = stmt.strip()
    if (not stmt or 
        stmt.startswith('--') or 
        stmt.upper().startswith('SET ') or 
        stmt.upper().startswith('CREATE DATABASE') or 
        stmt.upper().startswith('START TRANSACTION') or 
        stmt.upper().startswith('COMMIT')):
        continue
    cleaned_statements.append(stmt)


# Execute statements
for stmt in cleaned_statements:
    try:
        cursor.execute(stmt)
    except sqlite3.Error as e:
        print(f"Error executing statement:\n{stmt}\nError: {e}")

conn.commit()
conn.close()
print("SQL script executed successfully.")


SQL script executed successfully.


In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('chicago_data.db')
cursor = conn.cursor()

# Load and filter SQL script
with open("chicago_public_schools.sql", "r") as f:
    sql_script = f.read()

# Clean comments and MySQL-specific commands
cleaned_statements = []
for stmt in sql_script.split(';'):
    stmt = stmt.strip()
    if (not stmt or 
        stmt.startswith('--') or 
        stmt.upper().startswith('SET ') or 
        stmt.upper().startswith('CREATE DATABASE') or 
        stmt.upper().startswith('START TRANSACTION') or 
        stmt.upper().startswith('COMMIT')):
        continue
    cleaned_statements.append(stmt)


# Execute statements
for stmt in cleaned_statements:
    try:
        cursor.execute(stmt)
    except sqlite3.Error as e:
        print(f"Error executing statement:\n{stmt}\nError: {e}")

conn.commit()
conn.close()
print("SQL script executed successfully.")


In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('chicago_data.db')
cursor = conn.cursor()

# Load and filter SQL script
with open("chicago_crime.sql", "r") as f:
    sql_script = f.read()

# Clean comments and MySQL-specific commands
cleaned_statements = []
for stmt in sql_script.split(';'):
    stmt = stmt.strip()
    if (not stmt or 
        stmt.startswith('--') or 
        stmt.upper().startswith('SET ') or 
        stmt.upper().startswith('CREATE DATABASE') or 
        stmt.upper().startswith('START TRANSACTION') or 
        stmt.upper().startswith('COMMIT')):
        continue
    cleaned_statements.append(stmt)


# Execute statements
for stmt in cleaned_statements:
    try:
        cursor.execute(stmt)
    except sqlite3.Error as e:
        print(f"Error executing statement:\n{stmt}\nError: {e}")

conn.commit()
conn.close()
print("SQL script executed successfully.")
